## RandomForest Classifier

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


## Feature Engineering

In [3]:
# Handling Missing Values
# Handling Categorical Features
# Handling Outliers
# Feature Scaling

In [4]:
# Handling Missing Values
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [5]:
# EDA - There is no Missing Values

In [6]:
# Handling Categorical Features
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [7]:
# EDA - There is four categorical features, we have to convert it to numerical, via ENCODING

In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [9]:
df['sex'].unique()

['Female', 'Male']
Categories (2, object): ['Male', 'Female']

In [10]:
df['sex'].value_counts()

Male      157
Female     87
Name: sex, dtype: int64

In [11]:
# df['sex']=encoder.fit_transform(df['sex'])

In [12]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [13]:
df['sex'].value_counts()

Male      157
Female     87
Name: sex, dtype: int64

In [14]:
# Independent and Dependent Features
X = df.drop(['time'], axis=1)
y = df['time']

In [15]:
y.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [16]:
y.value_counts()

Dinner    176
Lunch      68
Name: time, dtype: int64

In [17]:
# Train, Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=0.20, shuffle=True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [19]:
df['smoker'].dtype

CategoricalDtype(categories=['Yes', 'No'], ordered=False)

In [20]:
categorical_features = ['smoker', 'day', 'sex']
numerical_features = ['total_bill', 'tip', 'size']

In [21]:
# Create the pipeline for Feature Engineering 
from sklearn.pipeline import Pipeline

In [22]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [23]:
# Steps of the process
steps = [
    ('imputation',SimpleImputer(strategy='mean')), # Handling Missing Values
    ('scaling', StandardScaler()), # Scale Numeric Features
    ('encoding', OneHotEncoder()) # Encode Categorical Features
]

In [24]:
# Create the PipeLine
pipeline = Pipeline(steps)

In [25]:
# Pipeline for Numerical Features
num_pipeline = Pipeline(
    steps = [
        ('imputation',SimpleImputer(strategy='median')), # Handling Missing Values
        ('scaling', StandardScaler()) # Scale Numeric Features
    ]
)


# Pipeline for Categorical Features
cat_pipeline = Pipeline(
    steps = [
        ('imputation',SimpleImputer(strategy='most_frequent')), # Handling Missing Values
        ('encoding', OneHotEncoder()) # Encode Categorical Features
    ]
)

In [26]:
# Combine Numeric and Categirical Pipeline
from sklearn.compose import ColumnTransformer

In [27]:
preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_features),
    ('cat_pipeline', cat_pipeline, categorical_features)
])

In [28]:
# Now perform the preprocessing
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [29]:
X_train

array([[-0.79306155, -0.2580329 , -0.61214068, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.46322744, -0.74211442, -0.61214068, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.80730659,  0.6399734 , -0.61214068, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-1.65383098, -1.46472887, -0.61214068, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.64749986,  0.32426806, -0.61214068, ...,  0.        ,
         0.        ,  1.        ],
       [ 2.75289699, -0.41237773,  0.45363997, ...,  0.        ,
         1.        ,  0.        ]])

In [30]:
# Model Training Automation

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

models = {
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier(),
    'SVC':SVC()
}


In [32]:
from sklearn.metrics import accuracy_score

def evaluate_model(X_train, y_train, X_test, y_test, models):
    report = {}
    
    for name,model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score = accuracy_score(y_test, y_pred)
        report[name]=score
        
    return report

In [33]:
evaluate_model(X_train, y_train, X_test, y_test, models)

{'Random Forest': 0.9591836734693877,
 'Decision Tree': 0.9387755102040817,
 'SVC': 0.9591836734693877}

In [35]:
# Hyperparameter Tuning

parameters = {
    'max_depth':[3,5,10,None],
    'n_estimators':[100,200,300],
    'criterion':['gini','entropy']
}

In [37]:
from sklearn.model_selection import RandomizedSearchCV
classifier = RandomForestClassifier()
clf = RandomizedSearchCV(classifier, param_distributions=parameters, cv=5, refit=True, scoring='accuracy', verbose=3)

In [38]:
clf.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.923 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=100;, score=1.000 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.949 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.923 total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.923 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=1.000 total time=   0.2s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.949 total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.949 tot

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [39]:
clf.best_params_

{'n_estimators': 100, 'max_depth': 10, 'criterion': 'entropy'}